In [79]:
import cv2
import json

vaga_atual = []
todas_vagas = []

def clique(event, x, y, flags, param):
    global vaga_atual, todas_vagas

    if event == cv2.EVENT_LBUTTONDOWN:
        vaga_atual.append((x, y))
    if len(vaga_atual) == 2:
        todas_vagas.append(vaga_atual.copy())
        cv2.rectangle(img_clone, vaga_atual[0], vaga_atual[1], (255, 0, 0), 2)
        print(f"Vaga salva: {vaga_atual}")
        vaga_atual = []


entrada = input('Nome do arquivo:')
img = cv2.imread(entrada)
img_clone = img.copy()
cv2.namedWindow("Imagem")
cv2.setMouseCallback("Imagem", clique)

while True:
    cv2.imshow("Imagem", img_clone)
    key = cv2.waitKey(1)
    if key == 27:  # ESC pra sair
        break

cv2.destroyAllWindows()

# Salvar coordenadas
with open("coordenadas_vagas.json", "w") as f:
    json.dump(todas_vagas, f)

print("Coordenadas salvas com sucesso!")


Vaga salva: [(7, 127), (62, 146)]
Vaga salva: [(5, 107), (60, 124)]
Vaga salva: [(77, 65), (135, 83)]
Vaga salva: [(76, 88), (135, 102)]
Vaga salva: [(75, 129), (135, 148)]
Coordenadas salvas com sucesso!


In [80]:
import numpy as np

def calcular_entropia(imagem):
    histograma = cv2.calcHist([imagem], [0], None, [256], [0, 256])
    histograma = histograma / histograma.sum()  # Normalizar
    entropia = -np.sum(histograma * np.log2(histograma + 1e-10))  # Evitar log(0)
    return entropia

LIMIAR_ENTROPIA = 6.3

img = cv2.imread(entrada)
with open("coordenadas_vagas.json", "r") as f:
    vagas = json.load(f)

for i, vaga in enumerate(vagas):
    (x1, y1), (x2, y2) = vaga
    vaga_crop = img[y1:y2, x1:x2]

    # Converter para escala de cinza
    cinza = cv2.cvtColor(vaga_crop, cv2.COLOR_BGR2GRAY)

    # Calcular a entropia da imagem recortada
    entropia = calcular_entropia(cinza)
    print(f"Vaga {i+1}: entropia = {entropia:.2f}")

    # Usar entropia para determinar se a vaga está livre ou ocupada
    if entropia < LIMIAR_ENTROPIA:
        status = "Livre"
        cor = (0, 255, 0)  # Verde
    else:
        status = "Ocupada"
        cor = (0, 0, 255)  # Vermelho

    # Desenhar o retângulo na imagem original
    cv2.rectangle(img, (x1, y1), (x2, y2), cor, 2)

# Mostrar imagem final com vagas marcadas
cv2.imshow("Resultado", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Vaga 1: entropia = 5.36
Vaga 2: entropia = 7.18
Vaga 3: entropia = 7.19
Vaga 4: entropia = 7.18
Vaga 5: entropia = 7.03
